In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# https://docs.google.com/forms/d/e/1FAIpQLSc-cevMFDA8fciVU5Smf42Ygt9CvXMQkRYaK_IiBp5EBcshBQ/formResponse - for answers

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [161]:
df_train = pd.read_csv('drive/MyDrive/MagolegoDS2022/ha3_train.csv')
df_test = pd.read_csv('drive/MyDrive/MagolegoDS2022/ha3_test.csv')
df_train.head(2)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,3472-OAOOR,Male,0,Yes,Yes,19,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,34.95,610.2,No
1,5393-RXQSZ,Male,0,No,No,1,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,No,Electronic check,79.60,79.6,Yes


1. What is the percentage of MonthlyCharges payed by customers who will Churn? 

In [ ]:
df_train.groupby('Churn')['MonthlyCharges'].agg(['sum'])

,sum
Churn,
No,190316.65
Yes,82466.20


In [ ]:
print (round(82466.20/(82466.20 + 190316.65)*100),'%')

30 %


2. Compute the profit of 20% discount (MonthlyCharges based) on a test set in scenario where you have "perfect", 100% accurate classification model. How much money company saves, assuming all customers accepts it and do not churn?

In [ ]:
# saved money = payments_with_accurate_discount - no_churn_payment = (churn_payment * 0.8 + no_churn_payment) - no_churn_payment = churn_payment * 0.8

df_test[df_test.Churn == "Yes"]['MonthlyCharges'].multiply(other = 0.8).sum(axis = 0)

45331.72

3. Build any binary classification model, which output probabilities (e.g. Logistic Regression, Gradient Boosting, etc.) using training set. Make predictions on a test set. Upload a csv file with your predictions (it should contain 2 columns, customerID and PredictedYes). For all remaining computations you must use these predictions.

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

In [5]:
# make dummies out of string variables
df_train = pd.get_dummies(df_train.drop(['customerID'], axis = 1), 
                        columns=['gender','Partner','Dependents','PaymentMethod'] + df_train.columns[6:17].tolist())

df_test = pd.get_dummies(df_test.drop(['customerID'], axis = 1), 
                        columns=['gender','Partner','Dependents','PaymentMethod'] + df_test.columns[6:17].tolist())

In [6]:
x_train, y_train = df_train.drop(columns = 'Churn'), df_train.Churn
x_test, y_test = df_test.drop(columns = 'Churn'), df_test.Churn

In [7]:
# need to convert predictors value into float64
cols = x_train.columns.tolist()
x_train[cols] = x_train[cols].apply(pd.to_numeric, errors='coerce', axis=1)

cols = x_test.columns.tolist()
x_test[cols] = x_test[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [8]:
# check for NaNs in training data
x_train.columns[x_train.isna().any()].tolist()

['TotalCharges']

In [9]:
x_train.TotalCharges[x_train.TotalCharges.isna() == True]

690    NaN
766    NaN
1128   NaN
1717   NaN
1904   NaN
1944   NaN
3221   NaN
Name: TotalCharges, dtype: float64

In [10]:
# inplace TotalCharges mean instead of NaN
imputer = SimpleImputer(strategy='mean')
imputer.fit(x_train.TotalCharges.values.reshape(-1, 1))
x_train.TotalCharges = imputer.transform(x_train.TotalCharges.values.reshape(-1, 1))

In [11]:
# check for NaNs in test data
x_test.columns[x_test.isna().any()].tolist()

['TotalCharges']

In [12]:
x_test.TotalCharges[x_test.TotalCharges.isna() == True]

565    NaN
1919   NaN
2380   NaN
2552   NaN
Name: TotalCharges, dtype: float64

In [13]:
imputer = SimpleImputer(strategy='mean')
imputer.fit(x_test.TotalCharges.values.reshape(-1, 1))
x_test.TotalCharges = imputer.transform(x_test.TotalCharges.values.reshape(-1, 1))

In [28]:
logreg = LogisticRegression(max_iter=10000, fit_intercept= False)

In [29]:
logreg.fit(x_train, y_train) # https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter

LogisticRegression(fit_intercept=False, max_iter=10000)

In [30]:
logreg.score(x_test,y_test)

0.8094393186657204

In [ ]:
#Upload a csv file with your predictions (it should contain 2 columns, customerID and PredictedYes). 
#For all remaining computations you must use these predictions.

In [72]:
yes_probab = logreg.predict_proba(x_test)[:,1] # [:,0] is "No" probability
test_customers = pd.read_csv('drive/MyDrive/MagolegoDS2022/ha3_test.csv', usecols = ["customerID"])

d = {'customerID': test_customers.customerID, 'PredictedYes': yes_probab}

predictions = pd.DataFrame(data = d)
predictions

,customerID,PredictedYes
0,8659-IOOPU,0.055082
1,0887-HJGAR,0.412609
2,1029-QFBEN,0.195818
3,7579-OOPEC,0.337537
4,8473-VUVJN,0.780902
...,...,...
2813,6476-EPYZR,0.117850
2814,0336-PIKEI,0.008743
2815,5201-CBWYG,0.005690
2816,0678-RLHVP,0.211548


In [73]:
from google.colab import files
predictions.to_csv('logregpredictions.csv', sep = ',')
#files.download('logregpredictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

4. What is ROC AUC of your model on a test set? Single integer number 0..100.


In [74]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix

In [75]:
y_train.replace(to_replace = ['Yes', 'No'], value = [1,0], inplace = True)
y_test.replace(to_replace = ['Yes', 'No'], value = [1,0], inplace = True)

In [76]:
y_pred_test = logreg.predict_proba(x_test)
y_pred_test

array([[0.9449183 , 0.0550817 ],
       [0.58739088, 0.41260912],
       [0.80418204, 0.19581796],
       ...,
       [0.99430958, 0.00569042],
       [0.7884524 , 0.2115476 ],
       [0.97987586, 0.02012414]])

In [77]:
print(roc_auc_score(y_true = y_test, y_score = y_pred_test[:, 1]))

0.8498801175596581


5. What are TP, FP, TN, FN for probability threshold 0.5 (4 integer numbers divided by a comma, e.g. 10,20,15,5). You can use example from https://docs.google.com/spreadsheets/d/132BCqQUFpPbmxPvdZCFbPkkXVmx5iLHQQZas9dWi4cs/edit?usp=sharing spreadsheet to compute the confusion matrix

In [80]:
threshold = 0.5

tn, fp, fn, tp = confusion_matrix(y_test, (y_pred_test[:, 1] > threshold).astype(int)).ravel()

print(f'TN: {tn}', f'FP: {fp}', f'FN: {fn}', f'TP: {tp}')

ValueError: ignored

6. Compute the profit of 20% discount (MonthlyCharges based), assuming you give it to all customers predicted as Churned (TP and FP). Single float number, with 2 significant digits, e.g. 10.16 .

In [83]:
# join "predictions" (id + predicted churn) with df_test.MonthlyCharges
charges_w_predicts = pd.read_csv('drive/MyDrive/MagolegoDS2022/ha3_test.csv', usecols = ["customerID","MonthlyCharges"])

charges_w_predicts['PredictedChurn'] = logreg.predict(x_test)
charges_w_predicts['p'] = y_pred_test.tolist()

In [85]:
charges_w_predicts['No_p'], charges_w_predicts['Yes_p'] = zip(*charges_w_predicts['p']) #https://stackoverflow.com/questions/40556830/pandas-dataframe-split-array-entry-into-two-columns
charges_w_predicts

,customerID,MonthlyCharges,PredictedChurn,p,No_p,Yes_p
0,8659-IOOPU,100.45,No,"[0.9449182995407163, 0.055081700459283775]",0.944918,0.055082
1,0887-HJGAR,45.70,No,"[0.5873908835274615, 0.41260911647253856]",0.587391,0.412609
2,1029-QFBEN,19.55,No,"[0.8041820396416738, 0.1958179603583262]",0.804182,0.195818
3,7579-OOPEC,50.15,No,"[0.6624633489027287, 0.3375366510972712]",0.662463,0.337537
4,8473-VUVJN,73.65,Yes,"[0.21909835811239353, 0.7809016418876065]",0.219098,0.780902
...,...,...,...,...,...,...
2813,6476-EPYZR,20.50,No,"[0.8821501585560956, 0.1178498414439044]",0.882150,0.117850
2814,0336-PIKEI,74.40,No,"[0.9912574235326832, 0.008742576467316758]",0.991257,0.008743
2815,5201-CBWYG,24.80,No,"[0.9943095813404221, 0.005690418659577873]",0.994310,0.005690
2816,0678-RLHVP,105.55,No,"[0.7884524012000887, 0.21154759879991125]",0.788452,0.211548


In [86]:
#Profit from discount: MonthlyCharges[Churn=Yes] * 0.8
charges_w_predicts[charges_w_predicts.PredictedChurn == "Yes"].MonthlyCharges.sum() * 0.8

40026.68000000001

7. What if we only give discount (20%) to top p% of customers (p% of customers with the highest probability to churn, predicted by our model). Find p, which maximizes the profit. Recall that p might not coincide with probability threshold outputed by your classifier (e.g. using threshold of 0.65 you end up with 50% of the customers). Single integer number 0..100.

In [137]:
#len(charges_w_predicts.Yes_p.sort_values(ascending = False)[0:3])/len(charges_w_predicts)*100 # percent of people with max p to Churn
charges_w_predicts[["MonthlyCharges","Yes_p"]].sort_values(by = "Yes_p",ascending = False)[0:1]

,MonthlyCharges,Yes_p
810,95.1,0.864075


In [133]:
data = pd.DataFrame(data = {'top_Percent_ChurnYes': [],'Profit':[]})

d = ['a','b']
d = pd.Series(data = d)
f = ['c','d']
f = pd.Series(data = f)
pd.concat([d, f])
# add value to dic 'top_Percent_ChurnYes'
# add value to the dic 'Profit'
# convert the dics to dfs
# concat dfs horizontally (axis = 1)
percent = {'top_Percent_ChurnYes': [1,2,3]}
profit = {'Profit':[3,4,5]}

percent = pd.DataFrame(percent)
profit = pd.DataFrame(profit)

pd.concat([percent,profit], axis = 1)

,top_Percent_ChurnYes,Profit
0,1,3
1,2,4
2,3,5


In [181]:
topped_data = charges_w_predicts.sort_values(by = "Yes_p",ascending = False)
percent_dic = {'top_Percent_ChurnYes': []}
profit_dic = {'Profit':[]}

for i in range(0, len(topped_data)):
  percent = len(topped_data[0:i])/len(topped_data)*100 # percent of people with max p to Churn
  # profit = charges[Churn==Yes]*discount - charges[Churn==No]*discount
  profit = topped_data[topped_data.PredictedChurn == "Yes"][0:i].MonthlyCharges.sum()*0.8 - topped_data[topped_data.PredictedChurn == "No"][0:i].MonthlyCharges.sum()*0.2
  percent_dic['top_Percent_ChurnYes'].append(percent)
  profit_dic['Profit'].append(profit)

percent_df = pd.DataFrame(percent_dic)
profit_df = pd.DataFrame(profit_dic)

pd.concat([percent_df,profit_df], axis = 1).sort_values(by = "Profit",ascending = False)


,top_Percent_ChurnYes,Profit
630,22.356281,30904.32
631,22.391767,30891.90
632,22.427253,30881.94
629,22.320795,30881.08
633,22.462740,30871.80
...,...,...
4,0.141945,241.15
3,0.106458,183.98
2,0.070972,117.33
1,0.035486,57.29


8. Instead of 20% discount we now give constant 10$ discount. How your answer to the previous question changes? Single integer number 0..100.

In [190]:
percent_dic = {'top_Percent_ChurnYes': []}
profit_dic = {'Profit':[]}

for i in range(0, len(topped_data)):
  percent = len(topped_data[0:i])/len(topped_data)*100 # percent of people with max p to Churn
  # profit = charges[Churn==Yes]*discount - charges[Churn==No]*discount
  profit = (topped_data[topped_data.PredictedChurn == "Yes"][0:i].MonthlyCharges - 10).sum() - (topped_data[topped_data.PredictedChurn == "No"][0:i].MonthlyCharges - 10).sum()
  percent_dic['top_Percent_ChurnYes'].append(percent)
  profit_dic['Profit'].append(profit)


percent_df = pd.DataFrame(percent_dic)
profit_df = pd.DataFrame(profit_dic)

pd.concat([percent_df,profit_df], axis = 1).sort_values(by = "Profit",ascending = False)


,top_Percent_ChurnYes,Profit
602,21.362669,4776.05
603,21.398155,4761.45
598,21.220724,4758.10
601,21.327182,4722.60
604,21.433641,4703.25
...,...,...
2397,85.060326,-67687.05
2398,85.095813,-67687.05
2399,85.131299,-67687.05
2400,85.166785,-67687.05
